In [ ]:
#pip install python-kasa flask requests picamera

In [ ]:
import sqlite3, datetime, time, queue, threading, sys, os, asyncio, kasa, pathlib, json, dateutil, requests, flask, logging # dhcp-lease-list

'''
with open('secret.json','w') as fw: 
    sec = {'bedroom_bulb_ip':"192.168.x.x",
           'bedroom_ac_ip'  :"192.168.x.x" }  
    json.dump(sec, fw)
'''
with open('secret.json') as fr: sec = json.load(fr) 

In [ ]:
pathlib.Path('log').mkdir(parents=True, exist_ok=True)
new_db = f'log/SHT40.{datetime.datetime.fromtimestamp(time.time()).strftime("%Y-%m-%dT%H:%M:%S")}.db' 

def database(): 
    con = sqlite3.connect(new_db)   
    cur = con.cursor() 
    cur.execute('CREATE TABLE IF NOT EXISTS log  (Temperature REAL, Humidity REAL, Photoresistor INTEGER, timestamp REAL);')
    cur = con.cursor() 
    while True: 
        opt, foo = q_in.get() # q.get() = opt_foo
        bar = False           # opt is a placeholder 
        try:
            if opt == 'insert':
                cur.execute('INSERT INTO log VALUES (?, ?, ?, ?);', 
                            (foo['Temperature (C)'], foo['Humidity (% rH)'], foo['Photoresistor'], time.time(),))  
            elif opt == 'sql':
                cur.execute(foo['sql'])  
                bar = cur.fetchall()
            elif opt == 'commit': 
                con.commit() 
                bar = 'Database commited'
                print('Database commited')
            elif opt == 'close': 
                con.commit()
                con.close()
                bar = 'Database commited and closed'
                print('Database commited and closed')
        except Exception as e:
            print('sql_exception', e, opt, foo)
            pass
        q_out.put(bar) 
        q_out.join()
        q_in.task_done()

q_in, q_out = queue.Queue(), queue.Queue() 
threading.Thread(target=database, name='thread-1', daemon=True).start() 

def database_operator(opt_foo): 
    q_in.join()        
    q_in.put(opt_foo)  
    bar = q_out.get()  
    q_out.task_done()   
    return bar

In [ ]:
UTC = dateutil.tz.gettz('UTC') 
class HUB:
    def __init__(self, temp_ceil = 28.5, lumi_ceil = 650):
        self.kasa_ac         = kasa.SmartPlug(sec['bedroom_ac_ip'])   
        self.kasa_bulb       = kasa.SmartBulb(sec['bedroom_bulb_ip'])  
        self.kasa_queue      = queue.Queue()
        self.params          = { 'temp_ceil':temp_ceil,
                                 'lumi_ceil':lumi_ceil, 
                                'night_mode':False,
                               'manual_mode':False,
                                'sunrise_dt':None,
                                'sunset_dt' :None, 
                                'bulb_timer':False # most of the time, bulb_timer is True. 
                                                   # It is false upon update and test begin. 
                               } 
        time.sleep(3)
        threading.Thread(target=self.kasa_thread).start()
        threading.Thread(target=self.update).start() 
        
    async def kasa_worker(self):
        while True: 
            device_name, method_name = self.kasa_queue.get() 
            device = getattr(  self, device_name)
            method = getattr(device, method_name)
            await method()
            self.kasa_queue.task_done()
                    
    def kasa_thread(self):
        asyncio.run(self.kasa_worker()) 

    def kasa_director(self, device_name, method_name):
        #kasa_queue.join()
        self.kasa_queue.put((device_name, method_name)) 
        
    def update(self):
        lat, lng = (40.7063311, -73.9971931)
        onehour = datetime.timedelta(hours=1) 
        while 1:
            print('[Daily Sunrise & Sunset Update]')
            date = datetime.datetime.now().strftime('%Y-%m-%d')
            r = requests.get(f'https://api.sunrise-sunset.org/json?lat={lat}&lng={lng}&date={date}&formatted=0')
            if r.json()['status'] != 'OK': raise
            self.params['sunrise_dt'] = datetime.datetime.fromisoformat(r.json()['results']['sunrise'])
            self.params['sunset_dt']  = datetime.datetime.fromisoformat(r.json()['results']['sunset']) 
            while 1: 
                if self.params['sunrise_dt'].day != datetime.datetime.now().day: 
                    print('A New Day Has Come')
                    break
                self.kasa_director(  'kasa_ac', 'update')
                self.kasa_director('kasa_bulb', 'update') 
                if datetime.datetime.now().astimezone(UTC) < self.params['sunrise_dt'] + onehour or \
                   datetime.datetime.now().astimezone(UTC) > self.params['sunset_dt'] - onehour: # Daylight
                    self.params['night_mode'] = True 
                else:                                                        # At night
                    self.params['night_mode'] = False
                time.sleep(60)
                if self.params['bulb_timer']: self.params.update({'bulb_timer':False})

hub = HUB()

In [ ]:
index_html_string = ''' <!DOCTYPE html>
<html>
  <head>
    <meta charset="utf-8">
    <title> hub </title>
    <style>
      .inp {height: 100px; width: 100px}  
      
      #slider {
        -webkit-appearance: none;
        width: 100%;
        height: 25px;
        background: #d3d3d3;
        outline: none;
        opacity: 0.7;
        -webkit-transition: .2s;
        transition: opacity .2s;
      }
      
      #slider:hover {
        opacity: 1;
      } 
      
      #slider::-webkit-slider-thumb {
        -webkit-appearance: none;
        appearance: none;
        width: 100px;
        height: 100px;
        background: #04AA6D;
        cursor: pointer;
      }
      
      #slider::-moz-range-thumb {
        width: 100px;
        height: 100px;
        background: #04AA6D;
        cursor: pointer;
      } 
      
      td {text-align: center; }
    </style>
  </head>
  <body> 
    <form method="POST" action="/" enctype="multipart/form-data"> 
      <table style="width: 500px;" > 
        <tr>
          <td class="inp">Light Mode:</td> 
          <td style="position: relative;">
            <input type="radio" name="manual_mode" value="1" {{'checked' if hub.params['manual_mode'] else ''}} class="inp">
            <span style="position: absolute;">Manu</span> 
          </td> 
          <td style="position: relative;">
            <input type="radio" name="manual_mode" value='0' {{'' if hub.params['manual_mode'] else 'checked'}} class="inp"> 
            <span style="position: absolute; top: 0px; left: 0px; ">Auto</span> 
          </td> 
        </tr>  
        <tr>
          <td>Max Celsius Degress:</td>
          <td><input type="text" name="temp_ceil" value="{{hub.params['temp_ceil']}}" style="font-size: 40px;" class="inp" id="output"/></td>
          <td><input type="submit" value="Submit" class="inp"></td>
        </tr>  
        <tr >
          <td colspan="3"><input type="range" min="1" max="100" value="{{ 10*hub.params['temp_ceil']-200}}" style="height:100px; width:100%" class="inp" id="slider"></td>
          <script>
            var output = document.getElementById("output");
            var slider = document.getElementById("slider");  
            slider.oninput = function() {
              output.setAttribute("value", 20+slider.value/10);   
            }
          </script> 
        </tr>
        <tr> 
        </tr>  
        <tr> 
          <td> <button type="button" onclick="light()" class="inp">Light</button> </td>
          <td> <button type="button" class="inp">Placeholder</button> </td>
          <td> <button type="button" class="inp">Placeholder</button> </td> 
          <script>
            function light() { 
              var xmlHttp = new XMLHttpRequest();
              xmlHttp.open( "GET", "/remote?pin=15", false ); 
              xmlHttp.send( null );
            }; 
             
          </script> 
        </tr>  
        <tr >
          <td colspan="3"> <img src="{{sec.cam_1_url}}"></img> </td>  
        </tr>
        <tr >
          <td colspan="3"> <img src="{{sec.cam_2_url}}"></img> </td> 
        </tr>
        
        <tr >
          <td>EA:DB:84:98:31:F4 </td> <td>MicroPython-9831f4 </td> <td> Controller</td> 
        </tr>
        <tr >
          <td>EA:DB:84:9C:54:9E </td> <td>MicroPython-9c549e </td> <td>unknown </td> 
        </tr>
        <tr >
          <td>EA:DB:84:9B:90:48 </td> <td>MicroPython-9b9048 </td> <td>HDMI Relay </td> 
        </tr>
        		  
        		  
        		  
        
      </table>  
      <br/> 
    </form>  
  </body>
</html> 
'''


log = logging.getLogger('werkzeug') 
log.setLevel(logging.ERROR) 

app = flask.Flask(__name__)  

@app.route("/", methods=['GET', 'POST'])
def index(): 
    req = flask.request 
    if req.method == 'GET':
        return flask.render_template_string(index_html_string, hub=hub, sec=sec) 
    elif req.method == 'POST':  
        hub.params['temp_ceil'] = float(req.form['temp_ceil']) 
        hub.params['manual_mode'] = int(req.form['manual_mode']) 
        return flask.redirect("/", code=302)
      
@app.route("/remote", methods=['GET'])
def remote(): 
    pin = flask.request.args.get('pin', -1) 
    if   pin == '13' :  
        if hub.kasa_ac.is_on:  
            hub.kasa_director('kasa_ac', 'turn_off')
            hub.kasa_director('kasa_ac', 'update')
            d = 'A/C Off' 
        else: 
            hub.kasa_director('kasa_ac', 'turn_on')
            hub.kasa_director('kasa_ac', 'update')
            d = 'A/C On'  
    elif pin == '15' : # 
        if hub.kasa_bulb.is_on:  
            hub.kasa_director('kasa_bulb', 'turn_off')
            hub.kasa_director('kasa_bulb', 'update')
            hub.params['manual_mode'] = True
            d = 'Light Off'
        else:   
            hub.kasa_director('kasa_bulb', 'turn_on')
            hub.kasa_director('kasa_bulb', 'update') 
            hub.params['manual_mode'] = False
            d = 'Light ON'
    return flask.jsonify({'response':d})

@app.route("/log", methods=['POST'])
def log():
    T = flask.request.json['Temperature (C)']
    P = flask.request.json['Photoresistor'] 
    if T > hub.params['temp_ceil'] and hub.kasa_ac.is_on == False: 
        hub.kasa_director('kasa_ac', 'turn_on')
        hub.kasa_director('kasa_ac', 'update')  
    elif T < hub.params['temp_ceil'] - 1 and hub.kasa_ac.is_on == True: 
        hub.kasa_director('kasa_ac', 'turn_off')
        hub.kasa_director('kasa_ac', 'update') 
        
    if hub.params['manual_mode'] or hub.params['bulb_timer']:
        pass 
    else:
        hub.params.update({'bulb_timer':True})
        if hub.params['night_mode']: 
            if P < hub.params['lumi_ceil']:  
                hub.kasa_director('kasa_bulb', 'turn_on')
                hub.kasa_director('kasa_bulb', 'update') 
            elif hub.kasa_bulb.is_on:  
                hub.kasa_director('kasa_bulb', 'turn_off')
                hub.kasa_director('kasa_bulb', 'update') 
        else: 
            hub.kasa_director('kasa_bulb', 'turn_off')
            hub.kasa_director('kasa_bulb', 'update') 
    print(flask.request.json)
    database_operator(('insert', flask.request.json)) 
    return 'OK' 

@app.route("/sql", methods=['POST'])
def sql():  
    return {'result':database_operator(('sql', flask.request.json)).__repr__() }
 
@app.route("/params", methods=['GET', 'POST'])
def params():   
    req = flask.request 
    if req.method == 'GET':
        return hub.params
    elif req.method == 'POST':  #  
        hub.params.update(req.json) 
        return {'return_code':100}

@app.route('/heartbeat', methods=['GET']) 
def heartbeat():
    database_operator(('commit', None))  
    return 'Cheers!'
 
app.run('0.0.0.0','8080')